In [1]:
import ccxt, pandas as pd, pathlib, datetime as dt

# ---------- settings ----------
symbols = ["BTC/USDT", "ETH/USDT"]
since   = ccxt.binance().parse8601("'2022-01-01T00:00:00Z'")  # 1 Jan 2020
out_dir = pathlib.Path("../data")
out_dir.mkdir(exist_ok=True)

exchange = ccxt.binance()

cols = ["timestamp","open","high","low","close","volume"]

def fetch_all_ohlcv(exchange, symbol, timeframe, since, limit=500):
    """
    Loop through the CCXT fetch_ohlcv endpoint until no more new bars
    come back. Returns one big list of [ts, open, high, low, close, vol].
    """
    all_bars = []
    while True:
        bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
        if not bars:
            break
        all_bars += bars
        # advance since to just after the last timestamp we got:
        since = bars[-1][0] + 1
        # if we received fewer than `limit` bars, we’re done
        if len(bars) < limit:
            break
    return all_bars



for pair in symbols:
    raw = fetch_all_ohlcv(exchange, pair, timeframe="1d", since=since, limit=500)
    df  = pd.DataFrame(raw, columns=cols)
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    # convert ms -> datetime, set UTC
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    asset = pair.split("/")[0].lower()          # "btc" or "eth"
    file  = out_dir / f"{asset}_usdt_1d.csv"
    df.to_csv(file, index=False)
    print(f"Saved → {file} ({len(df)} rows)")


Saved → ../data/btc_usdt_1d.csv (1276 rows)
Saved → ../data/eth_usdt_1d.csv (1276 rows)
